In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
from features.settings import Baseline_Settings, Pair_Hinge_Settings, Pair_Log_Settings, Triplet_Hinge_Settings, Triplet_Log_Settings
from features.models import ConvModelSmall, ConvModelMedium, ConvModelLarge
from features.models import GCNNModelSmall, GCNNModelMedium, GCNNModelLarge
from classes.models import LinearClassifier
from datasets import load_mnist_datasets, load_cifar10
from sklearn.svm import LinearSVC
import time

/home/dqi/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/dqi/.conda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data, samplers, pair_samplers, triplet_samplers = load_cifar10(augment=True)

CIFAR-10 loaded in 101 seconds


In [3]:
def test_model(feature_model, settings, data, num_features, num_classes, feature_samplers, classification_samplers,
               f_num_steps, f_lr, f_keep_prob, f_reg, c_num_steps, c_lr, c_reg):
    
    tf.reset_default_graph()
    
    f_train, f_valid, f_test = feature_samplers
    c_train, c_valid, c_test = classification_samplers
    (x_train, y_train), (x_valid, y_valid), (x_test, y_test) = data
    _, H, W, C = x_train.shape
    
    x = tf.placeholder(tf.float32, shape=[None, H, W, C])
    y = tf.placeholder(tf.int64, shape=[None])    
    dropout = tf.placeholder(tf.float32)
    training = tf.placeholder(tf.bool)
    f_model = feature_model(x=x, y=y, settings=settings, num_chan=C, num_features=num_features, 
              lr=f_lr, reg=f_reg, dropout=dropout, training=training)
    
    features = tf.placeholder(tf.float32, shape=[None, num_features])
    c_model = LinearClassifier(x=features, y=y, num_features=num_features, num_classes=num_features, 
                               lr=c_lr, reg=c_reg)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        print("begin training features // num_features: %g, lr: %g, reg: %g, dropout: %g" %(num_features, f_lr, f_reg, f_keep_prob))
        train_time = time.time()
        for step in range(f_num_steps):        
            x_, y_ = f_train.sample(700)
            sess.run(f_model.optimize, feed_dict={x:x_, y:y_, dropout:f_keep_prob, training:True})     
            if step % 1000 == 0:
                train_loss, train_acc = sess.run([f_model.loss, f_model.acc], 
                                                 feed_dict={x:x_, y:y_, dropout:1.0, training:False})
                print("\tstep %d: train loss %g, train error %g"%(step, train_loss, 1 - train_acc))  
        train_time = time.time() - train_time
        print("end training features // time elapsed: %.4f s"%(train_time))
        
        eval_test_time = time.time()
        x_, y_ = f_valid.sample(1000)
        test_error = 1 - sess.run(f_model.acc, feed_dict={x:x_, y:y_, dropout:1.0, training:False})
        eval_test_time = time.time() - eval_test_time
        print("validation set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))
        
        eval_test_time = time.time()
        x_, y_ = f_test.sample(1000)
        test_error = 1 - sess.run(f_model.acc, feed_dict={x:x_, y:y_, dropout:1.0, training:False})
        eval_test_time = time.time() - eval_test_time
        print("test set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))
        
        print("begin training classifier")
        train_time = time.time()
        for step in range(c_num_steps):        
            x_, y_ = c_train.sample(700)
            features_ = sess.run(f_model.features, feed_dict={x:x_, dropout:1.0, training:False})
            sess.run(c_model.optimize, feed_dict={features:features_, y:y_, x:x_, dropout:1.0, training:False})
            if step % 1000 == 0:
                train_loss, train_acc = sess.run([c_model.loss, c_model.acc], feed_dict={features:features_, y:y_})
                print("\tstep %d: train loss %g, train error %g"%(step, train_loss, 1 - train_acc))  
        train_time = time.time() - train_time
        print("end training classifier // time elapsed: %.4f s"%(train_time))
        
        eval_test_time = time.time()
        features_ = sess.run(f_model.features, feed_dict={x:x_valid, dropout:1.0, training:False})
        test_error = 1 - sess.run(c_model.acc, feed_dict={features:features_, y:y_valid})
        eval_test_time = time.time() - eval_test_time
        print("validation set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))
        
        eval_test_time = time.time()
        features_ = sess.run(f_model.features, feed_dict={x:x_test, dropout:1.0, training:False})
        test_error = 1 - sess.run(c_model.acc, feed_dict={features:features_, y:y_test})
        eval_test_time = time.time() - eval_test_time
        print("test set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))

In [4]:
test_model(feature_model=GCNNModelSmall, settings=Baseline_Settings, num_features=10, num_classes=10,
           data=data, feature_samplers=samplers, classification_samplers=samplers,           
           f_num_steps=70000, f_lr=0.01, f_keep_prob=0.5, f_reg=0.001, 
           c_num_steps=30000, c_lr=0.005, c_reg=0.01)

begin training features // num_features: 10, lr: 0.01, reg: 0.001, dropout: 0.5
	step 0: train loss 2.93011, train error 0.9
	step 1000: train loss 2.17987, train error 0.617143
	step 2000: train loss 1.95676, train error 0.558571
	step 3000: train loss 1.88843, train error 0.528571
	step 4000: train loss 1.84255, train error 0.535714
	step 5000: train loss 1.79822, train error 0.515714
	step 6000: train loss 1.6618, train error 0.48
	step 7000: train loss 1.71183, train error 0.491429
	step 8000: train loss 1.50597, train error 0.448571
	step 9000: train loss 1.47371, train error 0.434286
	step 10000: train loss 1.51822, train error 0.464286
	step 11000: train loss 1.54765, train error 0.458571
	step 12000: train loss 1.36541, train error 0.401429
	step 13000: train loss 1.32192, train error 0.391429
	step 14000: train loss 1.28613, train error 0.377143
	step 15000: train loss 1.22978, train error 0.367143
	step 16000: train loss 1.25001, train error 0.374286
	step 17000: train loss 1

In [5]:
test_model(feature_model=GCNNModelSmall, settings=Triplet_Hinge_Settings, num_features=20, num_classes=10,
           data=data, feature_samplers=triplet_samplers, classification_samplers=samplers,           
           f_num_steps=70000, f_lr=0.01, f_keep_prob=0.5, f_reg=0.001, 
           c_num_steps=40000, c_lr=0.01, c_reg=0.1)

begin training features // num_features: 20, lr: 0.01, reg: 0.001, dropout: 0.5
	step 0: train loss 1.53422, train error 0.504286
	step 1000: train loss 1.30936, train error 0.368571
	step 2000: train loss 1.23402, train error 0.344286
	step 3000: train loss 1.16887, train error 0.317143
	step 4000: train loss 1.17245, train error 0.352857
	step 5000: train loss 1.09434, train error 0.317143
	step 6000: train loss 1.06942, train error 0.301429
	step 7000: train loss 1.04514, train error 0.302857
	step 8000: train loss 0.992905, train error 0.288571
	step 9000: train loss 0.926068, train error 0.257143
	step 10000: train loss 0.933511, train error 0.284286
	step 11000: train loss 0.859604, train error 0.272857
	step 12000: train loss 0.875563, train error 0.252857
	step 13000: train loss 0.867234, train error 0.27
	step 14000: train loss 0.865351, train error 0.271429
	step 15000: train loss 0.823649, train error 0.264286
	step 16000: train loss 0.773278, train error 0.245714
	step 1700

In [6]:
test_model(feature_model=GCNNModelSmall, settings=Triplet_Hinge_Settings, num_features=20, num_classes=10,
           data=data, feature_samplers=triplet_samplers, classification_samplers=samplers,           
           f_num_steps=70000, f_lr=0.01, f_keep_prob=0.6, f_reg=0.001, 
           c_num_steps=40000, c_lr=0.01, c_reg=0.1)

begin training features // num_features: 20, lr: 0.01, reg: 0.001, dropout: 0.6
	step 0: train loss 1.52439, train error 0.461429
	step 1000: train loss 1.32681, train error 0.381429
	step 2000: train loss 1.28268, train error 0.358571
	step 3000: train loss 1.13089, train error 0.308571
	step 4000: train loss 1.07105, train error 0.275714
	step 5000: train loss 0.995504, train error 0.265714
	step 6000: train loss 1.00756, train error 0.284286
	step 7000: train loss 0.927377, train error 0.257143
	step 8000: train loss 0.945428, train error 0.277143
	step 9000: train loss 0.924185, train error 0.274286
	step 10000: train loss 0.899717, train error 0.281429
	step 11000: train loss 0.812311, train error 0.23
	step 12000: train loss 0.764345, train error 0.211429
	step 13000: train loss 0.759501, train error 0.235714
	step 14000: train loss 0.724082, train error 0.202857
	step 15000: train loss 0.697143, train error 0.197143
	step 16000: train loss 0.659486, train error 0.197143
	step 17